In [2]:
libs <- c(
    'RColorBrewer',
    'ggplot2',
    'xgboost',
    'glmnet',
    'dplyr',
    'tidyr',
    'pROC',
    'ROCR',
    'stringr',
    'caret',
    'caTools'
)

for (lib in libs) {
        if (!require(lib, character.only = TRUE, quietly = TRUE)) {
            install.packages(lib, repos='http://cran.us.r-project.org')
        }
}

(.packages())

source("my_R_functions/utility_functions.R")
source("my_R_functions/stat_functions.R")
source("my_R_functions/plot_functions.R")
source("/ssd/mrichard/github/BDDS/trenadb/src/utils.R")
source("/ssd/mrichard/github/BDDS/footprints/testdb/src/dbFunctions.R")

Loaded glmnet 2.0-12


Attaching package: ‘dplyr’

The following object is masked from ‘package:xgboost’:

    slice

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘tidyr’

The following object is masked from ‘package:Matrix’:

    expand

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following object is masked from ‘package:glmnet’:

    auc

The following objects are masked from ‘package:stats’:

    cov, smooth, var


Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



[1] "caTools"      "caret"        "lattice"      "stringr"      "ROCR"        
 [6] "gplots"       "pROC"         "tidyr"        "dplyr"        "glmnet"      
[11] "foreach"      "Matrix"       "xgboost"      "ggplot2"      "RColorBrewer"
[16] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[21] "methods"      "base"

Loading required package: DBI
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:pROC’:

    var

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:Matrix’:

    colMeans, colSums, rowMeans, rowSums, which

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, len

In [2]:
load("Rdata_files/motif_class_pairs.Rdata")
head(motif.class)

motif,TF,class,family
MA0001.1,AGL3,Other Alpha-Helix,MADS
MA0002.1,RUNX1,Ig-fold,Runt
MA0003.1,TFAP2A,Zipper-Type,Helix-Loop-Helix
MA0004.1,Arnt,Basic helix-loop-helix factors (bHLH),PAS domain factors
MA0005.1,AG,Other Alpha-Helix,MADS
MA0006.1,Ahr::Arnt,Basic helix-loop-helix factors (bHLH),PAS domain factors


Now we're going to load both datasets and combine them. In doing so, we'll also tag them with something to tell us what seed they came from...

In [3]:
load("/ssd/mrichard/data/all.TF.df.fimo.hint.well.seed16.annotated.9.Rdata")

In [4]:
full.16 <- all.TF.df.fimo.hint.well.annotated
rm(all.TF.df.fimo.hint.well.annotated)

In [5]:
load("/ssd/mrichard/data/all.TF.df.fimo.hint.well.seed20.annotated.9.Rdata")

In [6]:
full.20 <- all.TF.df.fimo.hint.well.annotated
rm(all.TF.df.fimo.hint.well.annotated)

Our 2 datasets are now loaded, we need to put them together; let's remember what they look like:

In [7]:
str(full.16)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	24720556 obs. of  39 variables:
 $ motifname                               : chr  "Mmusculus-jaspar2016-Nfe2l2-MA0150.2" "Mmusculus-jaspar2016-Bach1::Mafk-MA0591.1" "Hsapiens-jaspar2016-JUND(var.2)-MA0492.1" "Hsapiens-jaspar2016-ATF7-MA0834.1" ...
 $ chrom                                   : chr  "1" "1" "1" "1" ...
 $ start                                   : int  1677938 1677939 1828558 2255916 2255789 2255823 2255891 2255917 2255949 2255983 ...
 $ endpos                                  : int  1677952 1677953 1828572 2255929 2255803 2255837 2255905 2255931 2255963 2255997 ...
 $ strand                                  : chr  "+" "+" "-" "-" ...
 $ motifscore                              : num  13.35 11.71 9.73 8.03 13.72 ...
 $ pval                                    : num  1.13e-05 1.37e-05 9.18e-05 6.68e-05 1.48e-05 1.48e-05 1.48e-05 8.28e-06 1.48e-05 1.48e-05 ...
 $ sequence                                : chr  "CACTGTGACTCCGCA" "ACTGTGA

In [8]:
str(full.20)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	24720556 obs. of  39 variables:
 $ motifname                               : chr  "Mmusculus-jaspar2016-Nfe2l2-MA0150.2" "Mmusculus-jaspar2016-Bach1::Mafk-MA0591.1" "Hsapiens-jaspar2016-JUND(var.2)-MA0492.1" "Hsapiens-jaspar2016-ATF7-MA0834.1" ...
 $ chrom                                   : chr  "1" "1" "1" "1" ...
 $ start                                   : int  1677938 1677939 1828558 2255916 2255789 2255823 2255891 2255917 2255949 2255983 ...
 $ endpos                                  : int  1677952 1677953 1828572 2255929 2255803 2255837 2255905 2255931 2255963 2255997 ...
 $ strand                                  : chr  "+" "+" "-" "-" ...
 $ motifscore                              : num  13.35 11.71 9.73 8.03 13.72 ...
 $ pval                                    : num  1.13e-05 1.37e-05 9.18e-05 6.68e-05 1.48e-05 1.48e-05 1.48e-05 8.28e-06 1.48e-05 1.48e-05 ...
 $ sequence                                : chr  "CACTGTGACTCCGCA" "ACTGTGA

## Part 1: Just add them

First, let's try just stacking the data frames and removing the duplicates by taking the union

In [21]:
full.data <- dplyr::union(full.16, full.20)

In [22]:
str(full.data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	28912150 obs. of  39 variables:
 $ motifname                               : chr  "Hsapiens-jaspar2016-ZNF143-MA0088.2" "Hsapiens-jaspar2016-ZNF143-MA0088.2" "Hsapiens-jaspar2016-ZNF143-MA0088.2" "Hsapiens-jaspar2016-ZNF143-MA0088.2" ...
 $ chrom                                   : chr  "Y" "Y" "Y" "Y" ...
 $ start                                   : int  17944333 3242941 20943555 11099886 15339505 11139731 13387665 15515701 20731514 9209315 ...
 $ endpos                                  : int  17944348 3242956 20943570 11099901 15339520 11139746 13387680 15515716 20731529 9209330 ...
 $ strand                                  : chr  "-" "+" "-" "-" ...
 $ motifscore                              : num  10.56 2.24 7.84 9.65 2.07 ...
 $ pval                                    : num  6.84e-06 9.32e-05 1.71e-05 9.36e-06 9.77e-05 8.42e-05 3.57e-05 7.80e-05 1.19e-05 4.80e-05 ...
 $ sequence                                : chr  "CACCCTCGGTGCACTG" "TT

In [23]:
all.TF.df.fimo.hint.well.annotated <- full.data
save(all.TF.df.fimo.hint.well.annotated, file = "/ssd/mrichard/data/stacked.annotated.9.Rdata")
rm(all.TF.df.fimo.hint.well.annotated)

## Part 2: Adding Features

Alternatively, we can treat the 16 and 20 seed hits separately; thus, we can join the 2 dataframes together, being sure to keep the 2 sets of Wellington/Hint scores and fractions separate as well. 

First, let's cut down the dataset by removing anything we don't need as a key or as features (pvalue, sequence, chrom, start, endpos)

In [35]:
# Remove designated columns
sub.20 <- select(full.20, -pval, -sequence, -start, -endpos)
sub.16 <- select(full.16, -pval, -sequence, -start, -endpos)

Now designate our keys, AKA the things that we want to add

In [36]:
# Designate the key columns
my.keys <- setdiff(names(sub.20), c("h_max_score","w_min_score","h_frac","w_frac"))
my.keys

[1] "motifname"                               
 [2] "chrom"                                   
 [3] "strand"                                  
 [4] "motifscore"                              
 [5] "loc"                                     
 [6] "cs_hit"                                  
 [7] "gc_content"                              
 [8] "asinh_tss_dist"                          
 [9] "Basic helix-loop-helix factors (bHLH)"   
[10] "Basic leucine zipper factors (bZIP)"     
[11] "C2H2 zinc finger factors"                
[12] "Fork head / winged helix factors"        
[13] "Helix-Turn-Helix"                        
[14] "Heteromeric CCAAT-binding factors"       
[15] "High-mobility group (HMG) domain factors"
[16] "Homeo domain factors"                    
[17] "Ig-fold"                                 
[18] "MADS box factors"                        
[19] "Nuclear receptors with C4 zinc fingers"  
[20] "Other"                                   
[21] "Other Alpha-Helix"                       
[22] "Other C4 zinc finger-type factors"       
[23] "Paired box factors"                      
[24] "Rel homology region (RHR) factors"       
[25] "Runt domain factors"                     
[26] "SMAD/NF-1 DNA-binding domain factors"    
[27] "STAT domain factors"                     
[28] "Tryptophan cluster factors"              
[29] "Winged Helix-Turn-Helix"                 
[30] "Zinc-coordinating"                       
[31] "Zipper-Type"

Now we'll do a full outer join, adding the scores and fractions columns with the suffixes "_16" or "_20"

In [37]:
joined.data <- full_join(sub.20, sub.16, by = my.keys, suffix = c("_20","_16"))

In [38]:
str(joined.data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	29534972 obs. of  39 variables:
 $ motifname                               : chr  "Mmusculus-jaspar2016-Nfe2l2-MA0150.2" "Mmusculus-jaspar2016-Bach1::Mafk-MA0591.1" "Hsapiens-jaspar2016-JUND(var.2)-MA0492.1" "Hsapiens-jaspar2016-JUND(var.2)-MA0492.1" ...
 $ chrom                                   : chr  "1" "1" "1" "1" ...
 $ strand                                  : chr  "+" "+" "-" "-" ...
 $ motifscore                              : num  13.35 11.71 9.73 9.73 8.03 ...
 $ loc                                     : chr  "chr1:1677938-1677952" "chr1:1677939-1677953" "chr1:1828558-1828572" "chr1:1828558-1828572" ...
 $ cs_hit                                  : num  1 1 1 1 1 1 1 1 1 1 ...
 $ h_max_score_20                          : num  2096 2096 207 207 465 ...
 $ w_min_score_20                          : num  -33.02 -33.02 -5.39 -5.39 -34 ...
 $ h_frac_20                               : num  0.48 0.46 0.22 0.22 0.14 0.08 0.22 0.28 0.14 0.2 ...

In [39]:
# Save it
all.TF.df.fimo.hint.well.annotated <- joined.data
save(all.TF.df.fimo.hint.well.annotated, file = "/ssd/mrichard/data/joined.annotated.9.Rdata")
rm(all.TF.df.fimo.hint.well.annotated)

In [40]:
summary(joined.data)

  motifname            chrom              strand            motifscore    
 Length:29534972    Length:29534972    Length:29534972    Min.   :-9.449  
 Class :character   Class :character   Class :character   1st Qu.:10.427  
 Mode  :character   Mode  :character   Mode  :character   Median :11.539  
                                                          Mean   :11.580  
                                                          3rd Qu.:12.865  
                                                          Max.   :33.622  
     loc                cs_hit       h_max_score_20    w_min_score_20    
 Length:29534972    Min.   :0.0000   Min.   :    0.0   Min.   :-1000.00  
 Class :character   1st Qu.:0.0000   1st Qu.:    0.0   1st Qu.:    0.00  
 Mode  :character   Median :0.0000   Median :    0.0   Median :    0.00  
                    Mean   :0.1328   Mean   :  130.4   Mean   :  -10.24  
                    3rd Qu.:0.0000   3rd Qu.:    6.0   3rd Qu.:    0.00  
                    Max.   :1.0

## Part 3: Removing Footprints

We want to proactively answer the question:

* What would happen if you just used motifs instead of footprints? What makes footprints special?

To that end, we're going to take our joined dataset, which should have the maximum amount of training data, and simply strip off the HINT and Wellington columns. The removal of these 8 columns will leave us with a motif-only set. 

In [1]:
# If necessary, load the dataset
load("/ssd/mrichard/data/joined.annotated.9.Rdata")

In [12]:
# Remove the 2 columns from the joined dataset
motifs.only <- all.TF.df.fimo.hint.well.annotated %>% select(-starts_with("h_"), -starts_with("w_"))

In [13]:
dim(motifs.only); dim(all.TF.df.fimo.hint.well.annotated)

[1] 29534972       31

[1] 29534972       39

Looks like this worked; save the data frame for use later

In [14]:
save(motifs.only, file = "/ssd/mrichard/data/joined.motifs.only.9.Rdata")
rm(all.TF.df.fimo.hint.well.annotated)